In [ ]:
from __future__ import division  # Importing division from future module to ensure Python 3.x behavior
import os  # Importing the os module for operating system functions
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # Setting the visible CUDA devices to GPU 2
import numpy as np  # Importing numpy for numerical operations
import scipy  # Importing scipy library
import matplotlib.pyplot as plt  # Importing matplotlib for visualization
from sklearn.metrics import roc_curve  # Importing functions for ROC curve calculation
from sklearn.metrics import roc_auc_score  # Importing function for calculating Area Under ROC curve
from sklearn.metrics import confusion_matrix  # Importing function for confusion matrix calculation
from sklearn.metrics import precision_recall_curve  # Importing function for Precision-Recall curve calculation
from sklearn.metrics import accuracy_score  # Importing function for accuracy score calculation
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose
from keras.models import Model

# Define the model architecture
def BCDU_net_D3(input_tensor=None):
    if input_tensor is None:
        inputs = Input(shape=(256, 256, 3))
    else:
        inputs = input_tensor

    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conc1 = concatenate([inputs, conv1], axis=3)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conc1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conc2 = concatenate([pool1, conv2], axis=3)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conc2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conc3 = concatenate([pool2, conv3], axis=3)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conc3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conc4 = concatenate([pool3, conv4], axis=3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conc4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    return model

# Function to normalize the dataset
def dataset_normalized(imgs):
    imgs_normalized = np.empty(imgs.shape)
    imgs_std = np.std(imgs)
    imgs_mean = np.mean(imgs)
    imgs_normalized = (imgs-imgs_mean)/imgs_std
    for i in range(imgs.shape[0]):
        imgs_normalized[i] = ((imgs_normalized[i] - np.min(imgs_normalized[i])) / (np.max(imgs_normalized[i])-np.min(imgs_normalized[i])))*255
    return imgs_normalized

# Load the test data and masks
te_data    = np.load('data_test.npy')
te_mask = np.load('mask_test.npy')
te_mask  = np.expand_dims(te_mask, axis=3)

print('ISIC18 Dataset loaded')

# Normalize the test data
te_data2  = dataset_normalized(te_data)

# Define the model architecture
model = BCDU_net_D3()

# Load the trained weights of the model
model.load_weights('weight_isic18.keras')

# Make predictions on the test data
predictions = model.predict(te_data, batch_size=8, verbose=1)

# Reshape the predictions and masks for evaluation
y_scores = predictions.reshape(predictions.shape[0]*predictions.shape[1]*predictions.shape[2]*predictions.shape[3], 1)
y_true = te_mask.reshape(te_mask.shape[0]*te_mask.shape[1]*te_mask.shape[2]*te_mask.shape[3], 1)

# Binarize the predictions and masks
y_scores = np.where(y_scores>0.5, 1, 0)
y_true   = np.where(y_true>0.5, 1, 0)

# Output folder for saving results
output_folder = 'output/'

# Calculate and plot ROC curve
fpr, tpr, thresholds = roc_curve((y_true), y_scores)
AUC_ROC = roc_auc_score(y_true, y_scores)
print ("\nArea under the ROC curve: " +str(AUC_ROC))
roc_curve =plt.figure()
plt.plot(fpr,tpr,'-',label='Area Under the Curve (AUC = %0.4f)' % AUC_ROC)
plt.title('ROC curve')
plt.xlabel("FPR (False Positive Rate)")
plt.ylabel("TPR (True Positive Rate)")
plt.legend(loc="lower right")
plt.savefig(output_folder+"ROC.png")

# Calculate and plot Precision-Recall curve
precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
precision = np.fliplr([precision])[0] 
recall = np.fliplr([recall])[0]
AUC_prec_rec = np.trapz(precision,recall)
print ("\nArea under Precision-Recall curve: " +str(AUC_prec_rec))
prec_rec_curve = plt.figure()
plt.plot(recall,precision,'-',label='Area Under the Curve (AUC = %0.4f)' % AUC_prec_rec)
plt.title('Precision - Recall curve')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc="lower right")
plt.savefig(output_folder+"Precision_recall.png")

# Calculate and print confusion matrix, accuracy, specificity, sensitivity, and precision
threshold_confusion = 0.5
print ("\nConfusion matrix:  Custom threshold (for positive) of " +str(threshold_confusion))
y_pred = np.empty((y_scores.shape[0]))
for i in range(y_scores.shape[0]):
    if y_scores[i]>=threshold_confusion:
        y_pred[i]=1
    else:
        y_pred[i]=0
confusion = confusion_matrix(y_true, y_pred)
print (confusion)
accuracy = accuracy_score(y_true, y_pred)
print ("Accuracy: " +str(accuracy))
specificity = 0
if float(confusion[0,0]+confusion[0,1])!=0:
    specificity = float(confusion[0,0])/float(confusion[0,0]+confusion[0,1])
print ("Specificity: " +str(specificity))
sensitivity = 0
if float(confusion[1,1]+confusion[1,0])!=0:
    sensitivity = float(confusion[1,1])/float(confusion[1,1]+confusion[1,0])
print ("Sensitivity: " +str(sensitivity))
precision = 0
if float(confusion[1,1]+confusion[0,1])!=0:
    precision = float(confusion[1,1])/float(confusion[1,1]+confusion[0,1])
print ("Precision: " +str(precision))

# Calculate and print IoU (Intersection over Union)
intersection = np.logical_and(y_true, y_pred)
union = np.logical_or(y_true, y_pred)
iou_score = np.sum(intersection) / np.sum(union)
print ("\nIntersection over Union (IoU): " +str(iou_score))

# Calculate and print F1 score
F1_score = f1_score(y_true, y_pred, labels=None, average='binary', sample_weight=None)
print ("\nF1 score (F-measure): " +str(F1_score))

# Save the performance metrics to a file
file_perf = open(output_folder+'performances.txt', 'w')
file_perf.write("Area under the ROC curve: "+str(AUC_ROC)
                + "\nArea under Precision-Recall curve: " +str(AUC_prec_rec)
                + "\nIntersection over Union (IoU): " +str(iou_score)
                + "\nF1 score (F-measure): " +str(F1_score)
                +"\n\nConfusion matrix:"
                +str(confusion)
                +"\nAccuracy: " +str(accuracy)
                +"\nSensitivity: " +str(sensitivity)
                +"\nSpecificity: " +str(specificity)
                +"\nPrecision: " +str(precision)
                )
file_perf.close()

# # Save 10 results with error rate lower than threshold
# threshold = 300
# good_prediction = np.zeros([predictions.shape[0],1], np.uint8)
# id_m = 0
# for idx in range(predictions.shape[0]):
#     esti_sample = predictions[idx]
#     true_sample = te_mask[idx]
#     esti_sample = esti_sample.reshape(esti_sample.shape[0]*esti_sample.shape[1]*esti_sample.shape[2], 1)
#     true_sample = true_sample.reshape(true_sample.shape[0]*true_sample.shape[1]*true_sample.shape[2], 1)
#     er = np.sum(esti_sample != true_sample)
#     if er < threshold:
#        good_prediction[id_m] = idx    
#        id_m += 1   

# fig,ax = plt.subplots(10,3,figsize=[15,15])

# # Plot the results
# for idx in range(10):
#     ax[idx, 0].imshow(np.uint8(te_data[good_prediction[idx,0]]))
#     ax[idx, 1].imshow(np.squeeze(te_mask[good_prediction[idx,0]]), cmap='gray')
#     ax[idx, 2].imshow(np.squeeze(predictions[good_prediction[idx,0]]), cmap='gray')

plt.savefig(output_folder+'sample_results.png')

ISIC18 Dataset loaded
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step

Area under the ROC curve: 0.917855871944938

Area under Precision-Recall curve: 0.6710391249647416

Confusion matrix:  Custom threshold (for positive) of 0.5
[[58248  3900]
 [  344  3044]]
Accuracy: 0.93524169921875
Specificity: 0.937246572697432
Sensitivity: 0.898465171192444
Precision: 0.43836405529953915
